## Práctico 5: Volcano Crossing

In [2]:
from volcano_crossing_env import VolcanoCrossing

env = VolcanoCrossing(slip_prob=0.5)

In [3]:
env.reset()

'21'

### Jugar manualmente

In [4]:
def run_game():
        print("Play manually...")
        obs = env.reset()
        print(obs)
        done = False
        step_counter = 0
        all_rewards = 0
        env.render()

        while not done:
            action = input("Next action: ")
            env.check_action(action)
            obs, reward, done_env, _ = env.step(action)
            print(f'{obs=} {reward=} {done_env=}')
            all_rewards += reward
            done = done_env
            env.render()
            step_counter += 1

In [5]:
#run_game()

### Jugar con una policy

In [6]:
def policy_south(state) :
  return 'S'

In [29]:
def run(policy) :
  U = 0
  done = False
  state = env.reset()
  while not done:
    state, reward, done, _ = env.step(policy[state])
    U += reward
  return U

In [30]:
run(policies)

2

### Estimación de la policy por promedio

In [35]:
r = 0
N = 50_000
for i in range(N):
    r += run(policies)

print (f'Average reward: {r/N}')

Average reward: -2.50908


### Policy Evaluation

In [22]:
def policy_evaluation(policy, delta = 0.05, gamma = 0.999):
  V = { '11':0, '12':0, '13':0, '14':0, '21':0, '22':0, '23':0, '24':0, '31':0, '32':0, '33':0, '34':0 }
  V_new = { '11':0, '12':0, '13':0, '14':0, '21':0, '22':0, '23':0, '24':0, '31':0, '32':0, '33':0, '34':0 }
  max_diff = delta + 1

  while max_diff > delta:
    for s in env.observation_space:
      a = policy[s]
      q = 0
      for s_prime in env.P[s][a].keys():
        q += env.P[s][a][s_prime] * (env.R[s][a][s_prime] + gamma * V[s_prime])
      V_new[s] = q
    max_diff = 0
    for s in V.keys():
      max_diff = max(max_diff, abs(V[s]-V_new[s]))
    V = V_new.copy()
  return V

In [24]:
south_policy = { '11':'S', '12':'S', '13':'S', '14':'S', '21':'S', '22':'S', '23':'S', '24':'S', '31':'S', '32':'S', '33':'S', '34':'S' }
V = policy_evaluation(south_policy, delta = 0.05, gamma = 0.999)
print(V)

{'11': -10.097479836569216, '12': -25.02876509699797, '13': 0.0, '14': 0.0, '21': -5.153966059429463, '22': -21.73443636193771, '23': 0.0, '24': -21.539534697673236, '31': 0.0, '32': -16.790727065526983, '33': -30.83304348111256, '34': -25.988180160178665}


### Policy Improvement

In [12]:
policies = { '11':'S', '12':'S', '13':'S', '14':'S', '21':'S', '22':'S', '23':'S', '24':'S', '31':'S', '32':'S', '33':'S', '34':'S' }

In [34]:
# Realizar un paso de policy improvement
def policy_improvement(V,policies):
    actions = ['N', 'S', 'E', 'W']
    for s in env.observation_space:
        q = {a: 0 for a in actions}
        for a in actions:
            for s_prime in env.P[s][a].keys():
                q[a] += env.P[s][a][s_prime] * (env.R[s][a][s_prime] + 0.999 * V[s_prime])
        policies[s] = max(q, key=q.get)
    return policies

### Policy Iteration

In [25]:
# Realizar policy iteration y ver cuál es la mejor acción para cada estado
from collections import defaultdict

policies = { '11':'S', '12':'S', '13':'S', '14':'S', '21':'S', '22':'S', '23':'S', '24':'S', '31':'S', '32':'S', '33':'S', '34':'S' }
policies_eval_value = policy_evaluation(policies, delta = 0.05, gamma = 0.999)
while True:
    new_pol = policy_improvement(V, policies)
    new_pol_eval_value = policy_evaluation(new_pol, delta = 0.05, gamma = 0.999)
    if new_pol_eval_value == policies_eval_value:
        break
    policies = new_pol.copy()
    policies_eval_value = new_pol_eval_value.copy()
    
print("Final policy:")
for s in policies.keys():
    print(f"State {s}: {policies[s]}")


Final policy:
State 11: S
State 12: W
State 13: N
State 14: N
State 21: S
State 22: W
State 23: N
State 24: N
State 31: N
State 32: W
State 33: W
State 34: N
